## Load Important Libraries

In [1]:
pip install -q -U "tensorflow-text==2.8.*"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.9.0 requires pyarrow<6,>=1, but you have pyarrow 8.0.0 which is incompatible.
tfx-bsl 1.9.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,<3,>=1.15.5, but you have tensorflow 2.8.4 which is incompatible.
tensorflow-transform 1.9.0 requires pyarrow<6,>=1, but you have pyarrow 8.0.0 which is incompatible.
tensorflow-transform 1.9.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,<2.10,>=1.15.5, but you have tensorflow 2.8.4 which is incompatible.
tensorflow-serving-api 2.9.0 requires tensorflow<3,>=2.9.0, but you have tensorflow 2.8.4 which is incompatible.
tensorflow-io 0.21.0 requires tensorflow<2.7.0,>=2.6.0, but you have tensorflow 2.8.4 which is incompatible.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesyst

In [2]:
import numpy as np
import pandas as pd
import re
import string
from tqdm import tqdm

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
#from official.nlp import optimization  # to create AdamW optimizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        path = os.path.join(dirname, filename)

2022-11-18 00:42:00.654732: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2022-11-18 00:42:00.654792: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Load the data

In [3]:
df = pd.read_csv(path)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


## Data Preprocessing

In [5]:
def preprocess(vocab_lookup_table, example_text):
    
    # Normalize text
    tf_text.normalize_utf8(example_text)

    # Tokenize into words
    word_tokenizer = tf_text.WhitespaceTokenizer()
    tokens = word_tokenizer.tokenize(example_text)

    # Tokenize into subwords
    subword_tokenizer = tf_text.WordpieceTokenizer(
        vocab_lookup_table, token_out_type=tf.int64)
    subtokens = subword_tokenizer.tokenize(tokens).merge_dims(1, -1)

    # Apply padding
    padded_inputs = tf_text.pad_model_inputs(subtokens, max_seq_length=16)
    return padded_inputs

In [6]:
df.sentiment.value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [7]:
df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x=='positive' else 0)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


### Split Input and Target value

In [8]:
x = df.review
y = df.sentiment

## Split Train & Test data

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x,y, stratify = y)

In [10]:
y_train.value_counts()

1    18750
0    18750
Name: sentiment, dtype: int64

## Load BERT preprocessor & Encoder from  Tensordlow hub

In [11]:
bert_preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

2022-11-18 00:42:09.204287: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2022-11-18 00:42:09.204352: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-18 00:42:09.204385: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (94dae0f63463): /proc/driver/nvidia/version does not exist
2022-11-18 00:42:09.204646: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the

### Check the Text preprocessor with any custom text

In [12]:
Text =[
    'Amazing movie, super',
]

preprocessor = bert_preprocessor(Text)
preprocessor.keys()

dict_keys(['input_word_ids', 'input_mask', 'input_type_ids'])

### `CLS Amazing movie , super SEP`

In [13]:
print('\ninput_type_ids \n','Shape :',preprocessor['input_type_ids'].shape,'\nMatrix\n',preprocessor['input_type_ids'].numpy())
print('\ninput_word_ids \n','Shape :',preprocessor['input_word_ids'].shape,'\nMatrix\n',preprocessor['input_word_ids'].numpy())
print('\ninput_mask \n','Shape :',preprocessor['input_mask'].shape,'\nMatrix\n', preprocessor['input_mask'].numpy())


input_type_ids 
 Shape : (1, 128) 
Matrix
 [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

input_word_ids 
 Shape : (1, 128) 
Matrix
 [[ 101 6429 3185 1010 3565  102    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    

### Check the BERT Encoder value

In [14]:
Encoder = bert_encoder(preprocessor)
Encoder.keys()

dict_keys(['encoder_outputs', 'sequence_output', 'pooled_output', 'default'])

In [15]:
#Encoder['sequence_output']
#Encoder['default']
#Encoder['encoder_outputs']
Encoder['pooled_output'].shape
Encoder['pooled_output']

<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-0.8408587 , -0.42045587, -0.8464164 ,  0.65676934,  0.604676  ,
        -0.19671197,  0.8160436 ,  0.19800909, -0.72023517, -0.99994195,
        -0.46657524,  0.866401  ,  0.9649077 ,  0.39229771,  0.86678106,
        -0.65357465, -0.31811884, -0.5556684 ,  0.35521817, -0.37095663,
         0.72209895,  0.99992687,  0.22844192,  0.21341476,  0.46579152,
         0.96549284, -0.8290586 ,  0.880514  ,  0.9328118 ,  0.73696333,
        -0.59841436,  0.11375102, -0.9839795 , -0.29258388, -0.90887   ,
        -0.98650306,  0.30324864, -0.7102465 ,  0.01472446,  0.04887506,
        -0.84173554,  0.26598442,  0.9998738 , -0.06954353,  0.18741927,
        -0.14819263, -0.99999833,  0.11251041, -0.8233195 ,  0.82611984,
         0.7524895 ,  0.6246767 ,  0.15382712,  0.48116982,  0.37244958,
        -0.11545344,  0.00654148,  0.09571902, -0.20923522, -0.5517341 ,
        -0.4746632 ,  0.32082403, -0.8240155 , -0.87861735,  0.8624712 ,
 

## Build the Model

In [16]:
def build_model():
    text_input = tf.keras.layers.Input(shape =(), dtype = tf.string, name = 'text')
    preprocessor = bert_preprocessor(text_input)
    Encoder = bert_encoder(preprocessor)
    
    Dropout = tf.keras.layers.Dropout(0.3, name= 'dropout')(Encoder['pooled_output'])
    activation = tf.keras.layers.Dense(1,activation = 'sigmoid', name = 'output')(Dropout)
    
    model = tf.keras.Model(inputs = [text_input], outputs = [activation])
    model.compile(optimizer = 'adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])
    return model
model = build_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

### Train the model

In [17]:
model.fit(x_train, y_train, epochs = 3)

Epoch 1/3
1172/1172 [==============================] - 8721s 7s/step - loss: 0.6375 - accuracy: 0.6358
Epoch 2/3
1172/1172 [==============================] - 8742s 7s/step - loss: 0.5797 - accuracy: 0.6971
Epoch 3/3
1172/1172 [==============================] - 8734s 7s/step - loss: 0.5653 - accuracy: 0.7093


## Evaluation

In [18]:
loss, accuracy = model.evaluate(x_test, y_test)
print('The accuracy of model on unknown data is',round((accuracy*100),2),'%')

391/391 [==============================] - 2880s 7s/step - loss: 0.5253 - accuracy: 0.7528
The accuracy of model on unknown data is 75.28 %


In [19]:
# Predict the label
y_predicted = model.predict(x_test)

# Get the maximum value integer as the output for the predicted value 
label = {
    1:'positive',
    0:'Negative'
}
pred_labels = [label[np.argmax(i)] for i in y_predicted]
Actual = [label[i] for i in y_test]
print('Predicted Label :',pred_labels[:10])
print('Actual Label    :',Actual[:10])

Predicted Label : ['Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative']
Actual Label    : ['positive', 'Negative', 'Negative', 'Negative', 'positive', 'positive', 'Negative', 'Negative', 'Negative', 'Negative']


## Classification Report

In [20]:
print("Classification Report: \n", classification_report(Actual, pred_labels))

Classification Report: 
               precision    recall  f1-score   support

    Negative       0.50      1.00      0.67      6250
    positive       0.00      0.00      0.00      6250

    accuracy                           0.50     12500
   macro avg       0.25      0.50      0.33     12500
weighted avg       0.25      0.50      0.33     12500



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
Text =[
    'Amazing movie, super',
    'Vey Boring'
]

model.predict(Text)

array([[0.7600496],
       [0.3973584]], dtype=float32)